In [1]:
import pyximport
pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import gammapy
from gammapy.maps import Map
from gammapy.modeling import Fit,  Parameters, Covariance , Parameter
from gammapy.datasets import MapDataset ,Datasets
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    PointSpatialModel,
    GaussianSpatialModel,
    Models,
    FoVBackgroundModel,
)
import sys
sys.path.append('../../')
#from Dataset_Creation import sys_dataset
import Dataset_load 

FileNotFoundError: [Errno 2] No such file or directory: '/config.json'

In [ ]:
import os
os.getcwd()

In [ ]:
figformat = 'png'

In [ ]:
dataset_asimov = Dataset_load.create_asimov()

In [ ]:
edispKernelMap = dataset_asimov.edisp
edispKernel = edispKernelMap.get_edisp_kernel()
edispMap = edispKernelMap.edisp_map

In [ ]:
from astropy.visualization import quantity_support

def plot(edispKernel, ax=None, label_true_energy = True):
    ax = plt.gca() if ax is None else ax
    kwargs =dict()
    kwargs.setdefault("cmap", "GnBu")
    from matplotlib.colors import PowerNorm

    norm = PowerNorm(gamma=0.5, vmin=0, vmax=1)
    kwargs.setdefault("norm", norm)


    energy_axis = edispKernel.axes["energy"]
    if label_true_energy:
        energy_axis_true_ = edispKernel.axes["energy_true"]
    else:
        energy_axis_true_ = edispKernel.axes["energy"]

    with quantity_support():
        caxes = ax.pcolormesh(
            energy_axis_true_.edges, energy_axis.edges, edispKernel.data.T, **kwargs
        )

    label = "Probability density (A.U.)"
    ax.figure.colorbar(caxes, ax=ax, label=label)

    energy_axis_true_.format_plot_xaxis(ax=ax)
    energy_axis.format_plot_yaxis(ax=ax)
    return ax

In [ ]:
ax = plot(edispKernel)
ax.set_title(f'Edisp')
print(edispKernel.data.shape)
fig = plt.gcf()
fig.savefig("plots/1_Edisp_example."+figformat)

In [ ]:
energy = edispKernel.axes['energy']
energy_true = edispKernel.axes['energy_true']

In [ ]:
fig, axs = plt.subplots(2,1)

#Get energy resolution for a given true energy.
#The resolution is given as a percentage of the true energy


axs[0].plot(energy_true.center.value, edispKernel.get_resolution(energy_true.center), label = 'data')
axs[0].legend()
axs[0].set_xscale('log')
axs[0].set_title("Resolution")
axs[0].set_xlabel('True Energy [TeV]')
axs[0].set_ylabel('Resolution \n[% of true energy]')

axs[1].plot(energy_true.center.value, edispKernel.get_bias(energy_true.center), label = 'data')
axs[1].legend()
axs[1].set_xscale('log')
axs[1].set_title("Bias")
axs[1].set_xlabel('True Energy [TeV]')





plt.tight_layout()
fig.savefig("plots/1_Edisp_res_bias."+figformat)

In [ ]:
plt.errorbar(energy_true.center.value, edispKernel.get_mean(energy_true.center).value, 
             yerr =  edispKernel.get_resolution(energy_true.center).value, label = 'data')

plt.plot(energy_true.center.value, energy_true.center.value, color = 'grey')
plt.legend()
plt.loglog()
plt.xlabel("True Energy [TeV]")
plt.ylabel("Reco Energy [TeV]")
fig = plt.gcf()
fig.savefig("plots/1_Edisp_visuals."+figformat)

In [ ]:
from gammapy.maps import MapAxis
from gammapy.irf import EDispKernel, EDispKernelMap
def get_gaussian_kernel(bias, sigma, energy = energy):
    edisp_kernel = EDispKernel.from_gauss(
        energy_axis_true=energy.copy(name='energy_true'), 
        energy_axis=energy,
        sigma=1e-12+sigma,
        bias=bias
    )
    return edisp_kernel

In [ ]:


fig, axs = plt.subplots(2,2)
axs = axs.flatten()
delta = 1e-2
biass = [-0.1,0.0,0.1 ,0.2]
sigmass = [0]
gaussians = []
i = 0
for b in biass:
    for s in sigmass:
        print("b=", b, "s=", s)
        gaussian = get_gaussian_kernel(bias = b, sigma = s)
        print(gaussian.data.shape)
        gaussians.append(gaussian)
        plot(gaussian,axs[i], False)
        axs[i].set_title(f'bias={b}; res={s}')
        i +=1
print(gaussian.data.shape)
plt.tight_layout()
fig.savefig("plots/1_Gaussian_nbins1.svg")


In [ ]:
fig, axs = plt.subplots(2,1)

axs[0].plot(energy.center.value, edispKernel.get_resolution(energy.center), label = 'data',
           color = 'grey')
for i, c in enumerate(gaussians):
    axs[0].plot(energy.center.value, c.get_resolution(energy.center), label = f"bias={biass[i]}")
axs[0].legend()
axs[0].set_xscale('log')
axs[0].set_title("Resolution - Gaussian")
axs[0].set_xlabel("True Energy [TeV]")

plt.plot(energy.center.value, edispKernel.get_bias(energy.center), label = 'data',
        color = 'grey')
for i, c in enumerate(gaussians):
    plt.plot(energy.center.value, c.get_bias(energy.center), label = f"bias={biass[i]}")
plt.legend()
plt.xscale('log')
plt.title("Bias - Gaussian")
plt.tight_layout()
plt.xlabel("True Energy [TeV]")
fig.savefig("plots/1_Gaussian_res_bias_nbins1.svg")


In [ ]:

for i, c in enumerate(gaussians):

    plt.errorbar(energy_true.center.value, c.get_mean(energy_true.center).value, 
             yerr =  c.get_resolution(energy_true.center).value,  label = f"bias={biass[i]}")
plt.errorbar(energy_true.center.value, edispKernel.get_mean(energy_true.center).value, 
             yerr =  edispKernel.get_resolution(energy_true.center).value, label = 'data')
plt.plot(energy_true.center.value, energy_true.center.value, color = 'grey')
plt.legend()
plt.loglog()
plt.xlabel("True Energy [TeV]")
plt.ylabel("Reco Energy [TeV]")

In [ ]:
convs = []
fig, axs = plt.subplots(2,2)
axs = axs.flatten()

i = 0

for b in biass:
    for s in sigmass:
        gaussian = get_gaussian_kernel(bias = b, sigma = s)
        print(edispKernel.data.shape)
        print(gaussian.data.shape)
        
        conv = EDispKernel(
         axes = edispKernel.axes,
         data = np.matmul(edispKernel.data, gaussian.data)
        )
        plot(conv,axs[i])
        axs[i].set_title(f'bias={b}; res={s}')
        i +=1
        convs.append(conv)
plt.tight_layout()

fig.savefig("plots/1_Conv_nbins1.svg")


In [ ]:
fig, axs = plt.subplots(2,1)


for i, c in enumerate(convs):
    axs[0].plot(energy.center.value, c.get_resolution(energy.center), label = f"bias={biass[i]}")
    
axs[0].plot(energy.center.value, edispKernel.get_resolution(energy.center), label = 'data',)
axs[0].legend()
axs[0].set_xscale('log')
axs[0].set_title("Resolution")
axs[0].set_xlabel("True Energy [TeV]")



for i, c in enumerate(convs):
    plt.plot(energy.center.value, c.get_bias(energy.center), label = f"bias={biass[i]}")
plt.plot(energy.center.value, edispKernel.get_bias(energy.center), label = 'data',)
plt.legend()
plt.xscale('log')
plt.title("Bias")
plt.tight_layout()
plt.xlabel("True Energy [TeV]")

fig.savefig("plots/1_Conv_res_bias_nbins1.svg")


In [ ]:
plt.errorbar(energy_true.center.value, edispKernel.get_mean(energy_true.center).value, 
             yerr =  edispKernel.get_resolution(energy_true.center).value, label = 'data')
for i, c in enumerate(convs):
    plt.errorbar(energy_true.center.value, c.get_mean(energy_true.center).value, 
             yerr =  c.get_resolution(energy_true.center).value, label = f"bias={biass[i]}")

plt.plot(energy_true.center.value, energy_true.center.value, color = 'grey')
plt.legend()
plt.loglog()
plt.xlabel("True Energy [TeV]")
plt.ylabel("Reco Energy [TeV]")

In [ ]:
fig, axs = plt.subplots(3,1, figsize = (5,5))
idx = 9
for b in biass:
    for s in sigmass:
        gaussian = get_gaussian_kernel(bias = b, sigma = s)
        conv = EDispKernel( axes = edispKernel.axes,  data = np.matmul(edispKernel.data, gaussian.data))
        axs[0].plot(energy.center, gaussian.data[idx,:], label = f'bias={b}')
        axs[2].plot(energy.center , edispKernel.data[idx,:]-conv.data[idx,:], label = f'bias={b}')
        axs[1].plot(energy.center , conv.data[idx,:], 'x', label = f'bias={b}')
        
axs[1].plot(energy.center ,edispKernel.data[idx,:], label = 'origi', color = 'grey')
axs[0].set_title("Gaussian")
axs[1].set_title("Convolved")
axs[2].set_title("Abs. Difference")



for a in axs:
    a.legend(loc = 'center left')
    a.set_xlabel("True Energy [TeV]")
    a.set_xscale('log')
    a.set_xlim(0.1, 10)
    
plt.tight_layout()
fig.savefig("plots/1_Scan_nbins1.svg")


## Upsampling

In [ ]:
widths = (energy.edges[1:] - energy.edges[:-1]) * 0.5

conv_data = conv.data[idx,:] 
n_bins = 10
idx = 9
conv_rebins = np.repeat(conv_data, n_bins)

energy_rebins = MapAxis(nodes =np.logspace(np.log10(energy.center[0].value), 
                                           np.log10(energy.center[-1].value),
                                           n_bins *len(energy.center)),
                        node_type='center',
                        name = 'energy',
                       unit = 'TeV')

In [ ]:
edispKernel_re = EDispKernel( axes = [energy_rebins.copy(name = 'energy_true'), energy_rebins] , 
                   data = np.repeat(np.repeat(edispKernel.data, n_bins,axis =0), n_bins, axis =1))
plt.imshow(edispKernel_re.data)

In [ ]:
plt.plot(energy_rebins.center, edispKernel_re.data[idx * n_bins],'.',label= "Upsampled Edisp",
        color = 'red')
plt.errorbar(energy.center, edispKernel.data[idx],
            xerr =widths, fmt='+',  label = 'Original Edisp', color = 'black',markersize = 14)
plt.xscale('log')
plt.legend()
plt.xlabel("True Energy [TeV]")
fig = plt.gcf()
fig.savefig("plots/1_Upsample."+figformat)


In [ ]:
fig, axs = plt.subplots(3,1)
idx_re = idx * n_bins
gaussians_10 =[]
for b in biass:
    for s in sigmass:
        gaussian = get_gaussian_kernel(bias = b, sigma = s,
                                      energy = energy_rebins)
        gaussians_10.append(gaussian)
        conv = EDispKernel( axes = edispKernel_re.axes,  data = np.matmul(edispKernel_re.data, gaussian.data))
        axs[0].plot(energy_rebins.center, gaussian.data[idx_re,:], label = f'bias={b}')
        axs[1].plot(energy_rebins.center , conv.data[idx_re,:],  label = f'bias={b}')
        axs[2].plot(energy_rebins.center , edispKernel_re.data[idx_re,:]-conv.data[idx_re,:], label = f'bias={b}')
        
        
axs[1].plot(energy_rebins.center ,edispKernel_re.data[idx_re,:], label = 'data')

axs[0].set_title("Gaussian")
axs[1].set_title("Convolved")
axs[2].set_title("Abs. Difference")

for a in axs:
    a.legend(loc = 'center left')
    a.set_xlabel("True Energy [TeV]")
    a.set_xscale('log')
    a.set_xlim(0.1, 10)
    
plt.tight_layout()
fig.savefig("plots/1_Scan_nbins10.svg")


In [ ]:
fig, axs = plt.subplots(1,1)
axs = [axs]
delta = 1e-2
sigmass = [0]
gaussians = []
i = 0
for b in biass[:1]:
    for s in sigmass:
        print("b=", b, "s=", s)
        plot(gaussians_10[i],axs[i], False)
        axs[i].set_title(f'bias={b}; res={s}')
        i +=1
print(gaussian.data.shape)
plt.tight_layout()
fig.savefig("plots/1_Gaussian_nbins10.svg")


In [ ]:
fig, axs = plt.subplots(3,1)
final_kernels = []
for b in biass:
    for s in sigmass:
        gaussian = get_gaussian_kernel(bias = b, sigma = s,
                                      energy = energy_rebins)
        conv = EDispKernel( axes = edispKernel_re.axes,  data = np.matmul(edispKernel_re.data, gaussian.data))
        axs[0].plot(energy_rebins.center, gaussian.data[idx_re,:], label = f' Gaussian bias={b}')
        axs[1].plot(energy.center , conv.data[idx_re,:].reshape(-1, n_bins).mean(axis=1),
                    label = f'bias={b}')
        axs[2].plot(energy.center , edispKernel.data[int(idx),:]-conv.data[idx_re,:].reshape(-1, n_bins).mean(axis=1),
                    label = f'bias={b}')
        conv_rescale = EDispKernel( axes = edispKernel.axes, 
                                   data =conv.data.reshape((len(edispKernel.axes["energy"].center), 
                                       n_bins, 
                                       len(edispKernel.axes["energy"].center), 
                                       n_bins)).mean(axis=(1, 3)))
        final_kernels.append(conv_rescale)
axs[1].plot(energy_rebins.center ,edispKernel_re.data[idx_re,:], label = 'data')

for a in axs:
    a.legend(loc = 'center left')
    a.set_xlabel("True Energy [TeV]")
    a.set_xscale('log')
    a.set_xlim(0.1, 10)
axs[0].set_title(f'Rebinned with factor: {n_bins}')
plt.tight_layout()
fig.savefig("plots/1_Scan_rebined_nbins10.svg")



In [ ]:


fig, axs = plt.subplots(2,2)
axs = axs.flatten()
delta = 1e-2
biass = [-0.1,0.0,0.1 ,0.2]
sigmass = [0]
gaussians = []
i = 0
for b in biass:
    for s in sigmass:
        print("b=", b, "s=", s)
        plot(final_kernels[i],axs[i], False)
        axs[i].set_title(f'bias={b}; res={s}')
        i +=1
print(final_kernels[0].data.shape)
plt.tight_layout()

fig.savefig("plots/1_Conv_nbins10."+figformat)


In [ ]:

fig = plt.figure()

for i, c in enumerate(final_kernels):
    plt.errorbar(energy_true.center.value , c.get_mean(energy_true.center).value, 
             yerr =  c.get_resolution(energy_true.center).value,
                #xerr = widths.value,
                 label = f"bias={biass[i]}",
                #fmt='o', markersize=2,
                 capsize=6,
                elinewidth = 0.1)
plt.errorbar(energy_true.center.value, edispKernel.get_mean(energy_true.center).value, 
             label = 'data'
            )
plt.fill_between(energy_true.center.value,
                 edispKernel.get_mean(energy_true.center).value- edispKernel.get_resolution(energy_true.center).value,
                 edispKernel.get_mean(energy_true.center).value+ edispKernel.get_resolution(energy_true.center).value,
                 alpha = 0.2,
                color ='tab:purple')
plt.plot(energy_true.center.value, energy_true.center.value, color = 'grey')
plt.legend()
plt.loglog()
plt.xlabel("True Energy [TeV]")
plt.ylabel("Reco Energy [TeV]")

fig.savefig('plots/1_comparison2d.'+figformat)

In [ ]:
fig, axs = plt.subplots(2,1)


for i, c in enumerate(final_kernels):
    axs[0].plot(energy.center.value, c.get_resolution(energy.center), label = f"bias={biass[i]}")
axs[0].plot(energy.center.value, edispKernel.get_resolution(energy.center), label = 'data',
           color = 'tab:purple')
axs[0].legend()
axs[0].set_xscale('log')
axs[0].set_title("Resolution")
axs[0].set_xlabel('True Energy [TeV]')



for i, c in enumerate(final_kernels):
    axs[1].plot(energy.center.value, c.get_bias(energy.center), label = f"bias={biass[i]}")
axs[1].plot(energy.center.value, edispKernel.get_bias(energy.center), label = 'data',
        )    
    
axs[1].legend()
axs[1].set_xscale('log')
axs[1].set_title("Bias")
axs[1].set_xlabel('True Energy [TeV]')

plt.tight_layout()

fig.savefig("plots/1_Conv_res_bias_nbins10."+figformat)
